# Module 3b: AgentCore Deployment with Observability

In this module, you'll deploy your Student Analytics Agent to **Amazon Bedrock AgentCore** with full observability using GenAI semantic conventions for CloudWatch GenAI Observability Dashboard.

## What You'll Learn

- How the AgentCore observable agent uses GenAI semantic conventions for proper span/event capture
- How to deploy your agent with OpenTelemetry instrumentation
- How to view traces, spans, and events in CloudWatch GenAI Observability Dashboard

## GenAI Semantic Conventions

For spans and events to appear correctly in CloudWatch GenAI Observability Dashboard, they must use:

1. **Recognized tracer scopes**: `openinference.instrumentation.*`
2. **GenAI semantic convention attributes**:
   - `gen_ai.operation.name`: "invoke_agent" or "execute_tool"
   - `gen_ai.agent.name`: Agent identifier
   - `session.id`: Required for trace correlation
   - `gen_ai.usage.input_tokens`, `gen_ai.usage.output_tokens`: Token metrics

## Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    Amazon Bedrock AgentCore Runtime                     │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │                   Your Agent Container                          │    │
│  │  ┌───────────────────────────────────────────────────────────┐  │    │
│  │  │  opentelemetry-instrument python main_observable.py       │  │    │
│  │  │                                                           │  │    │
│  │  │  Tracer scope: openinference.instrumentation.claude_agent_sdk │  │
│  │  │                                                           │  │    │
│  │  │  GenAI semantic convention spans:                         │  │    │
│  │  │  └── invoke_agent student-analytics-agent                 │  │    │
│  │  │      ├── gen_ai.operation.name: "invoke_agent"            │  │    │
│  │  │      ├── gen_ai.agent.name, session.id                    │  │    │
│  │  │      ├── gen_ai.usage.input_tokens, output_tokens         │  │    │
│  │  │      └── execute_tool <tool_name> (child spans)           │  │    │
│  │  └───────────────────────────────────────────────────────────┘  │    │
│  └─────────────────────────────────────────────────────────────────┘    │
│                                     │ OTLP/HTTP                         │
│                              (Automatic Export)                         │
└─────────────────────────────────────────────────────────────────────────┘
                                     │
                                     ▼
┌─────────────────────────────────────────────────────────────────────────┐
│                    Amazon CloudWatch                                    │
│           ┌──────────────────────────────────────────────────────┐      │
│           │  GenAI Observability Dashboard                       │      │
│           │  - Agent sessions & GenAI semantic spans             │      │
│           │  - Tool usage with gen_ai.tool.name                  │      │
│           │  - Events (user_message, assistant_message, finish)  │      │
│           └──────────────────────────────────────────────────────┘      │
└─────────────────────────────────────────────────────────────────────────┘
```

## Prerequisites

- **Module 2 completed** - AgentCore deployment basics
- Docker installed and running
- Transaction Search enabled in CloudWatch

## Step 1: Setup Environment

In [ ]:
import os
import sys
import json
import subprocess
import boto3
import yaml
from pathlib import Path
from dotenv import load_dotenv

# Set up paths
workshop_root = Path("..").resolve()
sys.path.insert(0, str(workshop_root))

# Load environment
load_dotenv(workshop_root / ".env")

# Ensure Bedrock is used
os.environ["CLAUDE_CODE_USE_BEDROCK"] = "1"

region = os.getenv('AWS_REGION', 'us-east-1')
print(f"Workshop root: {workshop_root}")
print(f"AWS Region: {region}")

## Step 2: Enable Transaction Search

Transaction Search must be enabled before traces will appear in CloudWatch.

In [ ]:
print("Checking Transaction Search status...")
print("=" * 60)

try:
    result = subprocess.run(
        ['aws', 'xray', 'get-trace-segment-destination', '--region', region],
        capture_output=True, text=True
    )
    
    if result.returncode == 0:
        response = json.loads(result.stdout)
        destination = response.get('Destination', 'Unknown')
        status = response.get('Status', 'Unknown')
        
        print(f"Destination: {destination}")
        print(f"Status: {status}")
        
        if destination == 'CloudWatchLogs' and status == 'ACTIVE':
            print("\n✅ Transaction Search is ENABLED!")
        else:
            print("\n⚠️  Transaction Search needs to be enabled!")
            print(f"\nOpen this link to enable:")
            print(f"https://{region}.console.aws.amazon.com/cloudwatch/home?region={region}#xray:settings/transaction-search")
            print("\nSteps:")
            print("1. Click 'Edit'")
            print("2. Select 'Ingest spans as structured logs'")
            print("3. Click 'Save'")
            print("4. Wait ~10 minutes for it to become active")
except Exception as e:
    print(f"Error checking status: {e}")
    print(f"\nManually enable at:")
    print(f"https://{region}.console.aws.amazon.com/cloudwatch/home?region={region}#xray:settings/transaction-search")

## Step 3: Verify Requirements Include OpenTelemetry and OpenInference

The following packages must be in `requirements.txt`:
- `aws-opentelemetry-distro` - AWS distribution for OpenTelemetry auto-instrumentation
- `openinference-instrumentation-bedrock` - GenAI semantic conventions for Bedrock
- `openinference-semantic-conventions` - GenAI semantic convention definitions

In [ ]:
requirements_file = workshop_root / "requirements.txt"

print("Checking requirements.txt for OpenTelemetry and OpenInference...")
print("=" * 60)

with open(requirements_file, 'r') as f:
    content = f.read()

# Check for all required packages
required_packages = [
    ('aws-opentelemetry-distro', 'AWS OpenTelemetry auto-instrumentation'),
    ('openinference-instrumentation-bedrock', 'GenAI semantic conventions for Bedrock'),
    ('openinference-semantic-conventions', 'GenAI semantic convention definitions'),
]

all_found = True
for package, description in required_packages:
    if package in content:
        print(f"✅ {package}")
        print(f"   → {description}")
    else:
        print(f"❌ {package} NOT found")
        print(f"   → {description}")
        all_found = False

if not all_found:
    print("\n⚠️  Missing packages! Add them to requirements.txt:")
    print("   aws-opentelemetry-distro~=0.12.1")
    print("   openinference-instrumentation-bedrock>=0.1.0")
    print("   openinference-semantic-conventions>=0.1.0")
else:
    print("\n✅ All observability packages found!")

## Step 4: Verify AgentCore Configuration Has Observability Enabled

The AgentCore configuration should have observability enabled.

In [ ]:
config_file = workshop_root / ".bedrock_agentcore.yaml"

print("Checking AgentCore configuration...")
print("=" * 60)

if config_file.exists():
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    
    # Check observability setting
    agents = config.get('agents', {})
    for agent_name, agent_config in agents.items():
        aws_config = agent_config.get('aws', {})
        observability = aws_config.get('observability', {})
        enabled = observability.get('enabled', False)
        
        print(f"Agent: {agent_name}")
        print(f"  Observability enabled: {enabled}")
        
        if not enabled:
            print("\n⚠️  Observability is not enabled!")
            print("   Update .bedrock_agentcore.yaml to set:")
            print("   aws.observability.enabled: true")
        else:
            print("\n✅ Observability is enabled!")
else:
    print("❌ AgentCore configuration not found.")
    print("   Complete Module 2 first to configure AgentCore.")

## Step 5: Update Entrypoint for Observable Agent

For Module 3b, we use `main_observable.py` which imports the observable agent with GenAI semantic convention spans.

**File structure:**
```
main.py                    → agent/agent_agentcore.py      (Module 2: basic)
main_observable.py         → agent/agent_agentcore_observable.py (Module 3b: with spans)
```

### GenAI Semantic Convention Spans in agent_agentcore_observable.py

The observable agent creates spans using GenAI semantic conventions for CloudWatch compatibility:

**Tracer scope:** `openinference.instrumentation.claude_agent_sdk`

**Root span: `invoke_agent student-analytics-agent`**

| Attribute | Description |
|-----------|-------------|
| `gen_ai.operation.name` | "invoke_agent" |
| `gen_ai.system` | "claude-agent-sdk" |
| `gen_ai.agent.name` | Agent identifier |
| `gen_ai.prompt` | User query (truncated) |
| `gen_ai.agent.tools` | JSON array of available tools |
| `session.id` | Session identifier for correlation |
| `gen_ai.usage.input_tokens` | Input tokens used |
| `gen_ai.usage.output_tokens` | Output tokens used |
| `agent.cost_usd` | Estimated cost in USD |
| `agent.num_turns` | Number of conversation turns |

**Child spans: `execute_tool <tool_name>`**

| Attribute | Description |
|-----------|-------------|
| `gen_ai.operation.name` | "execute_tool" |
| `gen_ai.tool.name` | Tool name |
| `skill.name` | Skill name (for Skill tool) |
| `db.statement` | SQL query (for Athena tool) |
| `db.system` | "athena" (for Athena tool) |

**Events:**

| Event | Purpose |
|-------|---------|
| `gen_ai.user.message` | User query submitted |
| `gen_ai.assistant.message` | Assistant response |
| `gen_ai.agent.finish` | Agent completed successfully |
| `gen_ai.agent.error` | Agent encountered error |

## Step 5b: Configure Entrypoint for Observability

Before deploying, we need to update the Dockerfile and config to use the observable agent with OpenTelemetry instrumentation.

**Changes for Module 3b:**
- Entrypoint: `main_observable.py` (instead of `main.py`)
- CMD: `opentelemetry-instrument python main_observable.py`

In [ ]:
# Update Dockerfile CMD for observability deployment
import yaml
import re

dockerfile_path = workshop_root / "Dockerfile"
config_file = workshop_root / ".bedrock_agentcore.yaml"

print("Configuring for Module 3b (observability deployment)...")
print("=" * 60)

# Check if Dockerfile exists
if not dockerfile_path.exists():
    print("❌ Dockerfile not found!")
    print("   Please run Module 2 first to generate the Dockerfile.")
else:
    # Read current Dockerfile
    with open(dockerfile_path, 'r') as f:
        dockerfile_content = f.read()
    
    # Check if already using observable
    if 'opentelemetry-instrument' in dockerfile_content and 'main_observable.py' in dockerfile_content:
        print("✅ Dockerfile: Already configured for observability")
    else:
        print("📝 Updating Dockerfile CMD for observability...")
        
        # Replace the CMD line to use opentelemetry-instrument
        dockerfile_content = re.sub(
            r'CMD \["python", "main.py"\]',
            'CMD ["opentelemetry-instrument", "python", "main_observable.py"]',
            dockerfile_content
        )
        
        with open(dockerfile_path, 'w') as f:
            f.write(dockerfile_content)
        print("   ✅ Updated CMD to: opentelemetry-instrument python main_observable.py")

# Update config entrypoint
if config_file.exists():
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    
    agent_name = config.get('default_agent', 'student_analytics_agent')
    current_entrypoint = config['agents'][agent_name].get('entrypoint', '')
    
    if current_entrypoint == 'main_observable.py':
        print(f"✅ Config entrypoint: main_observable.py")
    else:
        print(f"📝 Updating config entrypoint from '{current_entrypoint}' to 'main_observable.py'")
        config['agents'][agent_name]['entrypoint'] = 'main_observable.py'
        with open(config_file, 'w') as f:
            yaml.dump(config, f, default_flow_style=False, sort_keys=False)
        print("   ✅ Config updated")
    
    # Ensure observability is enabled
    obs_enabled = config['agents'][agent_name].get('aws', {}).get('observability', {}).get('enabled', False)
    if not obs_enabled:
        print("📝 Enabling observability in config...")
        config['agents'][agent_name]['aws']['observability']['enabled'] = True
        with open(config_file, 'w') as f:
            yaml.dump(config, f, default_flow_style=False, sort_keys=False)
        print("   ✅ Observability enabled")
    else:
        print("✅ Observability: enabled in config")
else:
    print("❌ Config file not found. Complete Module 2 first.")

print("\n" + "=" * 60)
print("Module 3b deployment will use:")
print("  - main_observable.py → agent/agent_agentcore_observable.py")
print("  - OpenTelemetry auto-instrumentation via ADOT")
print("  - Custom spans for agent workflow tracking")
print("=" * 60)

## Step 6: Deploy to AgentCore

Deploy using `agentcore deploy`. The observability is enabled automatically via ADOT instrumentation.

In [ ]:
print("Deploying to AgentCore with Observability...")
print("=" * 70)
print("This will:")
print("  1. Build container with OpenTelemetry instrumentation")
print("  2. Push to ECR")
print("  3. Deploy to AgentCore Runtime")
print("  4. Enable automatic trace export to CloudWatch")
print("\nEstimated time: 5-10 minutes")
print("=" * 70)

proceed = True

if proceed:
    print("\nStarting deployment...\n")
    
    result = subprocess.run(
        ["agentcore", "deploy"],
        cwd=str(workshop_root),
        capture_output=True,
        text=True,
        env={**os.environ, 'PYTHONUNBUFFERED': '1'}
    )
    
    if result.stdout:
        print("STDOUT:")
        print("-" * 70)
        print(result.stdout)
    
    if result.stderr:
        print("\nSTDERR:")
        print("-" * 70)
        print(result.stderr)
    
    print("\n" + "=" * 70)
    if result.returncode == 0:
        print("✅ Deployment completed successfully!")
        print("\nObservability is now enabled. Traces will appear in CloudWatch.")
    else:
        print(f"❌ Deployment failed (exit code: {result.returncode})")
    print("=" * 70)
else:
    print("\nDeployment skipped.")

## Step 7: Invoke the Agent to Generate Traces

Use `--session-id` flag to pass the `runtimeSessionId` for:
- **Session continuity** - Same session ID maintains context across multiple invocations
- **Trace correlation** - AgentCore links all invocations with the same session ID
- Session ID must be at least 33 characters (UUID format recommended)

In [ ]:
import uuid

test_queries = [
    "How many students are currently enrolled?",
    "What is the average scholarship coverage rate for students in each major?",
]

# Generate ONE session ID for all queries - this is the runtimeSessionId
# AgentCore uses this to maintain context and link traces together
session_id = str(uuid.uuid4())

print("Invoking deployed agent to generate traces...")
print("=" * 70)
print(f"\n📍 Runtime Session ID (same for all queries): {session_id}")
print("=" * 70)

for i, query_text in enumerate(test_queries, 1):
    print(f"\nQuery {i}: {query_text}")
    print("-" * 50)
    
    # Pass query in payload, session_id via CLI flag
    payload = json.dumps({"query": query_text})
    
    try:
        # Use --session-id flag for runtimeSessionId (AgentCore's session management)
        result = subprocess.run(
            ["agentcore", "invoke", "--session-id", session_id, payload],
            cwd=str(workshop_root),
            capture_output=True,
            text=True,
            timeout=300
        )
        
        if result.returncode == 0:
            # Show truncated response
            output = result.stdout[:1000]
            print(output)
            if len(result.stdout) > 1000:
                print("... [truncated]")
            print("✅ Completed")
        else:
            print(f"❌ Error: {result.stderr[:300]}")
            
    except subprocess.TimeoutExpired:
        print("⚠️ Timed out")
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)
print("✅ Queries completed. Traces will be available in CloudWatch within 1-2 minutes.")
print(f"\n📍 Filter traces by runtimeSessionId: {session_id}")

## Step 8: View Traces in CloudWatch GenAI Observability Dashboard

In [ ]:
# Get agent name from config
agent_name = 'student_analytics_agent'
if config_file.exists():
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    agent_name = config.get('default_agent', 'student_analytics_agent')

print("CloudWatch GenAI Observability Dashboard")
print("=" * 70)

base_url = f"https://{region}.console.aws.amazon.com/cloudwatch"

print(f"\n📊 GenAI Observability Dashboard (Agents):")
print(f"   {base_url}/home?region={region}#/gen-ai-observability/agent-core/agents")

print("\n" + "=" * 70)
print("\n📌 How to find your traces:")
print(f"   1. Open the GenAI Observability Dashboard link above")
print(f"   2. Look for your agent: '{agent_name}'")
print(f"   3. Click on the agent to see runtime metrics")
print(f"   4. Go to 'Sessions View' tab to see individual invocations")
print(f"   5. Go to 'Trace View' tab to see trace details")
print(f"   6. Click on a trace to see the span waterfall")

print("\n📌 Finding traces by Session ID:")
print("   1. In Trace View, look for traces with matching session.id attribute")
print("   2. The session.id is set in OTEL baggage and appears in span attributes")



## Cleanup (Optional)

In [ ]:
# Cleanup options

print("Cleanup Options:")
print("=" * 60)

# Option 1: Restore original entrypoint
print("\n1. Restore original entrypoint (main.py for Module 2):")
print("   Run the cell below to restore.")

# Option 2: Delete the deployed agent
print("\n2. Delete the deployed agent:")
print(f"   cd {workshop_root} && agentcore delete")

# Uncomment below to restore original entrypoint
# config_file = workshop_root / ".bedrock_agentcore.yaml"
# if config_file.exists():
#     with open(config_file, 'r') as f:
#         config = yaml.safe_load(f)
#     default_agent = config.get('default_agent', 'student_analytics_agent')
#     config['agents'][default_agent]['entrypoint'] = 'main.py'
#     with open(config_file, 'w') as f:
#         yaml.dump(config, f, default_flow_style=False, sort_keys=False)
#     print("✅ Restored entrypoint to main.py")

In [ ]:
# Uncomment to delete the deployed agent
# !cd {workshop_root} && agentcore delete

print("To cleanup the deployed agent, run:")
print(f"  cd {workshop_root} && agentcore delete")

---

*Workshop: Build Agentic AI Applications with Claude Agent SDK and Amazon Bedrock AgentCore*